<a href="https://colab.research.google.com/github/Ishikaaa/PDF-extraction/blob/main/PDF_extraction_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install langchain
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu
!pip install -U langchain-community
# !pip install datasets
# !pip install transformers
# !pip install accelerate

In [2]:
# import libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# import libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

# from langchain.llms import HuggingFacePipeline
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
import re
import pickle
import os


# Step-1: get text from single OR multiple PDFs
def get_pdf_text(pdf_docs):
    """
    args:
        pdf_docs: list of pdfs
    """
    text = ""

    # iterate through all pdfs
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        # iterate through all pages
        for page in pdf_reader.pages:
            text += page.extract_text()

    return text

def clean_context(context):
    # Remove non-text elements
    context = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', context)  # Remove email addresses
    context = re.sub(r'\bhttps?:\/\/\S+\b', '', context)  # Remove URLs
    context = re.sub(r'\b\d+\b', '', context)  # Remove standalone digits
    # Add more regex patterns to remove other non-text elements

    # Normalize whitespace
    context = re.sub(r'\s+', ' ', context)

    # Lowercase text
    context = context.lower()

    return context

# Step-2: get the text chunks
def get_text_chunks(text):
    """
    : return
        a list of chunks of text that we will feed to our model
    """

    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_text(text)
    return chunks


# Step-3: Create Vector store
def get_vectorstore(text_chunks):
    # instructor embeddings
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def train_model(model_name):
    model_name = "t5-large"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Create a pipeline for text2text generation
    generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    return generator


def step5(vector_store, generator, questions):
    for question in questions:
        docs = vector_store.similarity_search(question)
        context = docs[0].page_content

        # Combine the question and context for T5
        input_text = f"question: {question} context: {context}. Provide a detailed answer."

        # Generate the answer with specific parameters
        result = generator(input_text, max_length=150, num_beams=4, early_stopping=True)

        # Decode the generated text
        answer = result[0]['generated_text']

        # print(f"Context: {context}")
        print(f"Question: {question}")
        print(f"Answer: {answer}")

In [ ]:
import os
if __name__ == "__main__":
    # Step-1 Load PDF
    pdf_docs = ["NIPS-2017-attention-is-all-you-need-Paper.pdf"]
    raw_text = get_pdf_text(pdf_docs)

    clean_text = clean_context(raw_text)

    # Step-2: get the text chunks
    text_chunks = get_text_chunks(clean_text)


    # Step-3
    vector_store = get_vectorstore(text_chunks)
    # print("text_chunks: ", vector_store)

    # Step-4
    model_filename = 'PDF_extraction_model.pkl'
    if os.path.exists(model_filename):
        with open(model_filename, 'rb') as file:
            generator = pickle.load(file)
        print("Model loaded successfully!")
    else:
        model_name = "t5-large"
        # model_name = "t5-small"
        generator = train_model(model_name)
        with open(model_filename, 'wb') as file:
            pickle.dump(generator, file)
        print("Model saved successfully!")

    # # Step-5
    # questions = ["Who is Aidan N. Gomez", "What is encoder and decoder?", "What did Ying Cao do?"]
    # step5(vector_store, generator, questions)

load INSTRUCTOR_Transformer
max_seq_length  512


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
import time

def step5(vector_store, generator, questions):
    for question in questions:
        begin = time.time()
        docs = vector_store.similarity_search(question)
        # context = docs[0].page_content
        context = " ".join([doc.page_content for doc in docs])

        # Combine the question and context for T5
        input_text = f"question: {question} context: {context}. Provide a detailed answer."

        # Generate the answer with specific parameters
        # result = generator(input_text, max_length=150, num_beams=4, early_stopping=True)
        result = generator(input_text, max_length=150, num_beams=4)

        # Decode the generated text
        answer = result[0]['generated_text']

        end = time.time()
        # print(f"Context: {context}")
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"runtime: {end - begin}")
        print("#########################")

# while True:
#     question = input("Question (type 'exit' to quit): ")
#     if question.lower() == 'exit':
#         print("Goodbye!")
#         break
#     else:
#         step5(vector_store, generator, question)
questions = ["Who is Aidan N. Gomez?", "Who is Noam Shazeer?", "What is encoder and decoder stacks?", "Who wrote Journal of Machine Learning Research?", "Who all were there in International Conference on Learning Representations?", "What is encoder?"]
# questions = ["Who all were there in International Conference on Learning Representations?"]
step5(vector_store, generator, questions)
# 55.93510174751282

In [ ]:
print(vector_store)

In [ ]:
import time

def step5(vector_store, generator, questions):
    for question in questions:
        begin = time.time()
        docs = vector_store.similarity_search(question)
        # context = docs[0].page_content
        context = " ".join([doc.page_content for doc in docs])

        # Combine the question and context for T5
        input_text = f"question: {question} context: {context}. Provide a detailed answer."

        # Generate the answer with specific parameters
        # result = generator(input_text, max_length=150, num_beams=4, early_stopping=True)
        result = generator(input_text, max_length=150, num_beams=5)

        # Decode the generated text
        answer = result[0]['generated_text']

        end = time.time()
        # print(f"Context: {context}")
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"runtime: {end - begin}")
        print("#########################")

# while True:
#     question = input("Question (type 'exit' to quit): ")
#     if question.lower() == 'exit':
#         print("Goodbye!")
#         break
#     else:
#         step5(vector_store, generator, question)
questions = ["Who is Aidan N. Gomez?", "Who is Noam Shazeer?", "What is encoder and decoder stacks?", "Who wrote Journal of Machine Learning Research?", "Who all were there in International Conference on Learning Representations?", "What is encoder?"]
# questions = ["Who all were there in International Conference on Learning Representations?"]
step5(vector_store, generator, questions)
# 55.93510174751282

In [10]:
import time

def step5(vector_store, generator, questions):
    for question in questions:
        begin = time.time()
        docs = vector_store.similarity_search(question)
        # context = docs[0].page_content
        context = " ".join([doc.page_content for doc in docs])

        # Combine the question and context for T5
        input_text = f"question: {question} context: {context}. Provide a detailed answer."

        # Generate the answer with specific parameters
        result = generator(input_text, max_length=150, num_beams=4, early_stopping=True)
        # result = generator(input_text, max_length=150, num_beams=4)

        # Decode the generated text
        answer = result[0]['generated_text']

        end = time.time()
        # print(f"Context: {context}")
        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"runtime: {end - begin}")
        print("#########################")

# while True:
#     question = input("Question (type 'exit' to quit): ")
#     if question.lower() == 'exit':
#         print("Goodbye!")
#         break
#     else:
#         step5(vector_store, generator, question)
questions = ["Who is Aidan N. Gomez?", "Who is Noam Shazeer?", "What is encoder and decoder stacks?", "Who wrote Journal of Machine Learning Research?", "Who all were there in International Conference on Learning Representations?", "What is encoder?"]
# questions = ["Who all were there in International Conference on Learning Representations?"]
step5(vector_store, generator, questions)
# 55.93510174751282

Question: Who is Aidan N. Gomez?
Answer: University of Toronto aidan@cs.toronto.edu
runtime: 52.0031213760376
#########################
Question: Who is Noam Shazee?
Answer: Noam Shazeer
runtime: 42.0838737487793
#########################
Question: What is encoder and decoder stacks?
Answer: N= 6 identical layers
runtime: 35.78837966918945
#########################
Question: Who wrote Journal of Machine Learning Research?
Answer: Jonas Gehring
runtime: 46.74137854576111
#########################
Question: Who all were there in International Conference on Learning Representations?
Answer: ukasz Kaiser and Ilya Sutskever
runtime: 53.18675422668457
#########################
Question: What is encoder?
Answer: maps an input sequence of symbol representations
runtime: 37.842244386672974
#########################


In [6]:
import time

def step5(vector_store, generator, questions):
    # for question in questions:
    begin = time.time()
    docs = vector_store.similarity_search(questions)
    # context = docs[0].page_content
    context = " ".join([doc.page_content for doc in docs])

    # Combine the question and context for T5
    input_text = f"question: {question} context: {context}. Provide a detailed answer."

    # Generate the answer with specific parameters
    # result = generator(input_text, max_length=150, num_beams=4, early_stopping=True)
    result = generator(input_text, max_length=150, num_beams=4)

    # Decode the generated text
    answer = result[0]['generated_text']

    end = time.time()
    # print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"runtime: {end - begin}")
    print("#########################")

while True:
    question = input("Question (type 'exit' to quit): ")
    if question.lower() == 'exit':
        print("Goodbye!")
        break
    else:
        step5(vector_store, generator, question)
# questions = ["Who is Aidan N. Gomez", "What is encoder and decoder stacks?", "Who wrote Journal of Machine Learning Research?", "Who all were there in International Conference on Learning Representations?", "What is encoder?"]
# questions = ["Who all were there in International Conference on Learning Representations?"]
# step5(vector_store, generator, questions)
# 55.93510174751282

Question (type 'exit' to quit): Who is Noam Shazee
Question: Who is Noam Shazee
Answer: Ashish Vaswani Google Brain avaswani@google.comNoam Shazeer Google Brain noam@google.comNiki Parmar Google Research nikip@google.com
runtime: 105.4311945438385
#########################
Question (type 'exit' to quit): Who is Noam Shazee?
Question: Who is Noam Shazee?
Answer: Noam Shazeer Google Brain noam@google.comNiki Parmar Google Research nikip@google.comAidan N. Gomezy University of Toronto aidan@cs.toronto.edu
runtime: 88.47019076347351
#########################
Question (type 'exit' to quit): Who is Illia Polosukhin?
Question: Who is Illia Polosukhin?
Answer: illia.polosukhin@gmail.com
runtime: 58.511088848114014
#########################
Question (type 'exit' to quit): exit
Goodbye!


In [ ]:
# Answer: noam@google.comNiki Parmar Google Research nikip@google.comAidan N. Gomezy University of Toronto aidan@cs.toronto.edu
# runtime: 95.47286462783813
# Answer: illia.polosukhin@gmail.com
# runtime: 54.402446031570435
# Answer: Noam Shazeer Google Brain noam@google.comNiki Parmar Google Research nikip@google.comAidan N. Gomezy University of Toronto aidan@cs.toronto.edu
# runtime: 112.03104734420776
# Question (type 'exit' to quit): Who is Aidan N. Gomez?
# Answer: University of Toronto aidan@cs.toronto.edu
# runtime: 73.178555727005

In [ ]:
## num_beans = 4
# #######################
# Question: Who is Aidan N. Gomez
# Answer: University of Toronto aidan@cs.toronto.edu
# runtime: 70.98402190208435
# #######################
# Question: What is encoder and decoder stacks?
# Answer: point-wise, fully connected layersWHo is
# runtime: 38.34173512458801
# #######################
# Question: Who wrote Journal of Machine Learning Research?
# Answer: Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin
# runtime: 53.56891655921936
# #######################
# Question: Who all were there in International Conference on Learning Representations?
# Answer: ukasz Kaiser and Ilya Sutskever
# runtime: 55.72374773025513

# ## num_beans = 5
# #######################
# Question: Who is Aidan N. Gomez
# Answer: University of Toronto aidan@cs.toronto.edu
# runtime: 80.05771541595459
# #######################
# Question: What is encoder and decoder stacks?
# Answer: point-wise, fully connected layers
# runtime: 53.74897265434265
# #######################
# Question: Who wrote Journal of Machine Learning Research?
# Answer: Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin
# runtime: 59.80621600151062
# #######################
# Question: Who all were there in International Conference on Learning Representations?
# Answer: ukasz Kaiser and Ilya Sutskever
# runtime: 73.25919771194458
# #######################
# Question: What is encoder?
# Answer: maps an input sequence of symbol representations (x1;:::;x n)to a sequence of continuous representations z= (z1;:::;z m)
# runtime: 60.467962980270386
